In [1]:
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import keras as k
import data
import os
import csv
import re
from string import digits

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.corpus import wordnet

In [13]:
# Define the file we want to read
MSFT_Sentiment_FILE = '8y_clean_msft_tweets_v.csv'
data_path = os.path.join(os.getcwd(), MSFT_Sentiment_FILE)

data_in = pd.read_csv(data_path, delimiter = ',', index_col=0)
print(data_in.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 474 entries, 0 to 473
Columns: 1989 entries, 2008/08/08 to 2016/07/01
dtypes: object(1989)
memory usage: 7.2+ MB
None


In [14]:
data_in.sample(n=5)

,2008/08/08,2008/08/11,2008/08/12,2008/08/13,2008/08/14,2008/08/15,2008/08/18,2008/08/19,2008/08/20,2008/08/21,...,2016/06/20,2016/06/21,2016/06/22,2016/06/23,2016/06/24,2016/06/27,2016/06/28,2016/06/29,2016/06/30,2016/07/01
412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
133,Georgia,neither,IOC,two,ack,accused,Putin,British,A,The,...,Grips,harsh,agency,disease,News,irth,routes,promote,cells,viral
247,And,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,taxes,Australian,Russian,company,Tayyip,weakening,offering,Yemen,sent,Duterte
237,Report,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,launches,...,soon,Brussels,Hyperloop,laws,EU,The,Temple,prison,trafficking,Vietnam
338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,possile,police,figures,NaN,NaN,NaN,NaN,NaN,NaN,UK


In [15]:
#TO get Bags of words
from collections import Counter
date_BOW = {}
for i in data_in.columns:
    cnt = Counter()
    for j in data_in[i]:
        cnt[j] += 1
    date_BOW[i] = cnt

In [16]:
print(data_in.columns)

Index(['2008/08/08', '2008/08/11', '2008/08/12', '2008/08/13', '2008/08/14',
       '2008/08/15', '2008/08/18', '2008/08/19', '2008/08/20', '2008/08/21',
       ...
       '2016/06/20', '2016/06/21', '2016/06/22', '2016/06/23', '2016/06/24',
       '2016/06/27', '2016/06/28', '2016/06/29', '2016/06/30', '2016/07/01'],
      dtype='object', length=1989)


In [20]:
print(type(date_BOW['2013/12/05']))
print(len(date_BOW['2013/12/05']))
print(len(date_BOW['2013/12/06']))

<class 'collections.Counter'>
286
234


In [22]:
date_BOW['2013/12/05'].keys() #The NaN should be removed!

dict_keys(['NSA', 'tracking', 'cellphone', 'locations', 'worldwide', 'Snowden', 'documents', 'show', 'Thieves', 'stole', 'coalt', 'likely', 'dead', 'dying', 'radiation', 'poisoning', 'Pope', 'sets', 'ody', 'child', 'ause', 'Francis', 'Vatican', 'committee', 'fight', 'Catholic', 'Church', 'Images', 'Where', 'Children', 'Sleep', 'Around', 'World', 'Paints', 'A', 'Powerful', 'Picture', 'Inequality', 'US', 'Emassy', 'aggressively', 'locked', 'minimum', 'wage', 'increase', 'Haitian', 'workers', 'This', 'past', 'summer', 'Army', 'egan', 'investigating', 'military', 'spent', 'nearly', 'million', 'construct', 'wellappointed', 'squarefoot', 'headquarters', 'southwestern', 'Afghanistan', 'commanders', 'area', 'want', 'never', 'een', 'used', 'Uruguay', 'could', 'ecome', 'first', 'country', 'world', 'nationalise', 'legalise', 'marijuana', 'On', 'Tuesday', 'Senate', 'vote', 'ill', 'would', 'make', 'sale', 'cannais', 'adults', 'legal', 'governmentcontrolled', 'If', 'passed', 'price', 'expected', 'e'

In [23]:
#remove nan
for date in date_BOW.keys():
    j = 0
    del_list = []
    for i in date_BOW[date].keys():
        if isinstance(i, float):
            j+=1
            del_list.append(i)
    for k in range(len(del_list)):        
        del date_BOW[date][del_list[k]]
    print(date,'done! cleared: ', j)

2008/08/08 done! cleared:  1
2008/08/11 done! cleared:  1
2008/08/12 done! cleared:  1
2008/08/13 done! cleared:  1
2008/08/14 done! cleared:  1
2008/08/15 done! cleared:  1
2008/08/18 done! cleared:  1
2008/08/19 done! cleared:  1
2008/08/20 done! cleared:  1
2008/08/21 done! cleared:  1
2008/08/22 done! cleared:  1
2008/08/25 done! cleared:  1
2008/08/26 done! cleared:  1
2008/08/27 done! cleared:  1
2008/08/28 done! cleared:  1
2008/08/29 done! cleared:  1
2008/09/02 done! cleared:  1
2008/09/03 done! cleared:  1
2008/09/04 done! cleared:  1
2008/09/05 done! cleared:  1
2008/09/08 done! cleared:  1
2008/09/09 done! cleared:  1
2008/09/10 done! cleared:  1
2008/09/11 done! cleared:  1
2008/09/12 done! cleared:  1
2008/09/15 done! cleared:  1
2008/09/16 done! cleared:  1
2008/09/17 done! cleared:  1
2008/09/18 done! cleared:  1
2008/09/19 done! cleared:  1
2008/09/22 done! cleared:  1
2008/09/23 done! cleared:  1
2008/09/24 done! cleared:  1
2008/09/25 done! cleared:  1
2008/09/26 don

2010/05/07 done! cleared:  1
2010/05/10 done! cleared:  1
2010/05/11 done! cleared:  1
2010/05/12 done! cleared:  1
2010/05/13 done! cleared:  1
2010/05/14 done! cleared:  1
2010/05/17 done! cleared:  1
2010/05/18 done! cleared:  1
2010/05/19 done! cleared:  1
2010/05/20 done! cleared:  1
2010/05/21 done! cleared:  1
2010/05/24 done! cleared:  1
2010/05/25 done! cleared:  1
2010/05/26 done! cleared:  1
2010/05/27 done! cleared:  1
2010/05/28 done! cleared:  1
2010/06/01 done! cleared:  1
2010/06/02 done! cleared:  1
2010/06/03 done! cleared:  1
2010/06/04 done! cleared:  1
2010/06/07 done! cleared:  1
2010/06/08 done! cleared:  1
2010/06/09 done! cleared:  1
2010/06/10 done! cleared:  1
2010/06/11 done! cleared:  1
2010/06/14 done! cleared:  1
2010/06/15 done! cleared:  1
2010/06/16 done! cleared:  1
2010/06/17 done! cleared:  1
2010/06/18 done! cleared:  1
2010/06/21 done! cleared:  1
2010/06/22 done! cleared:  1
2010/06/23 done! cleared:  1
2010/06/24 done! cleared:  1
2010/06/25 don

2012/05/01 done! cleared:  1
2012/05/02 done! cleared:  1
2012/05/03 done! cleared:  1
2012/05/04 done! cleared:  1
2012/05/07 done! cleared:  1
2012/05/08 done! cleared:  1
2012/05/09 done! cleared:  1
2012/05/10 done! cleared:  1
2012/05/11 done! cleared:  1
2012/05/14 done! cleared:  1
2012/05/15 done! cleared:  1
2012/05/16 done! cleared:  1
2012/05/17 done! cleared:  1
2012/05/18 done! cleared:  1
2012/05/21 done! cleared:  1
2012/05/22 done! cleared:  1
2012/05/23 done! cleared:  1
2012/05/24 done! cleared:  1
2012/05/25 done! cleared:  1
2012/05/29 done! cleared:  1
2012/05/30 done! cleared:  1
2012/05/31 done! cleared:  1
2012/06/01 done! cleared:  1
2012/06/04 done! cleared:  1
2012/06/05 done! cleared:  1
2012/06/06 done! cleared:  1
2012/06/07 done! cleared:  1
2012/06/08 done! cleared:  1
2012/06/11 done! cleared:  1
2012/06/12 done! cleared:  1
2012/06/13 done! cleared:  1
2012/06/14 done! cleared:  1
2012/06/15 done! cleared:  1
2012/06/18 done! cleared:  1
2012/06/19 don

2014/04/29 done! cleared:  1
2014/04/30 done! cleared:  1
2014/05/01 done! cleared:  1
2014/05/02 done! cleared:  1
2014/05/05 done! cleared:  1
2014/05/06 done! cleared:  1
2014/05/07 done! cleared:  1
2014/05/08 done! cleared:  1
2014/05/09 done! cleared:  1
2014/05/12 done! cleared:  1
2014/05/13 done! cleared:  1
2014/05/14 done! cleared:  1
2014/05/15 done! cleared:  1
2014/05/16 done! cleared:  1
2014/05/19 done! cleared:  1
2014/05/20 done! cleared:  1
2014/05/21 done! cleared:  1
2014/05/22 done! cleared:  1
2014/05/23 done! cleared:  1
2014/05/27 done! cleared:  1
2014/05/28 done! cleared:  1
2014/05/29 done! cleared:  1
2014/05/30 done! cleared:  1
2014/06/02 done! cleared:  1
2014/06/03 done! cleared:  1
2014/06/04 done! cleared:  1
2014/06/05 done! cleared:  1
2014/06/06 done! cleared:  1
2014/06/09 done! cleared:  1
2014/06/10 done! cleared:  1
2014/06/11 done! cleared:  1
2014/06/12 done! cleared:  1
2014/06/13 done! cleared:  1
2014/06/16 done! cleared:  1
2014/06/17 don

2016/04/22 done! cleared:  1
2016/04/25 done! cleared:  1
2016/04/26 done! cleared:  1
2016/04/27 done! cleared:  1
2016/04/28 done! cleared:  1
2016/04/29 done! cleared:  1
2016/05/02 done! cleared:  1
2016/05/03 done! cleared:  1
2016/05/04 done! cleared:  1
2016/05/05 done! cleared:  1
2016/05/06 done! cleared:  1
2016/05/09 done! cleared:  1
2016/05/10 done! cleared:  1
2016/05/11 done! cleared:  1
2016/05/12 done! cleared:  1
2016/05/13 done! cleared:  1
2016/05/16 done! cleared:  1
2016/05/17 done! cleared:  1
2016/05/18 done! cleared:  1
2016/05/19 done! cleared:  1
2016/05/20 done! cleared:  1
2016/05/23 done! cleared:  1
2016/05/24 done! cleared:  1
2016/05/25 done! cleared:  1
2016/05/26 done! cleared:  1
2016/05/27 done! cleared:  1
2016/05/31 done! cleared:  1
2016/06/01 done! cleared:  1
2016/06/02 done! cleared:  1
2016/06/03 done! cleared:  1
2016/06/06 done! cleared:  1
2016/06/07 done! cleared:  1
2016/06/08 done! cleared:  1
2016/06/09 done! cleared:  1
2016/06/10 don

In [24]:
#Set all nan -> '0'
def set_nan_0(tweet_df_col):
    for i in range(tweet_df_col.shape[0]):
        if isinstance(tweet_df_col[i], float):
            tweet_df_col[i] = ''
    return tweet_df_col

In [25]:
# Remove NaN
del_list = []
for date in date_BOW.keys():
    for key in date_BOW[date].keys():
        if isinstance(key, float):
            del_list.append(key)
    for i in range(len(del_list)):
        del date_BOW[date][i]
    print(date, 'Done!')


2008/08/08 Done!
2008/08/11 Done!
2008/08/12 Done!
2008/08/13 Done!
2008/08/14 Done!
2008/08/15 Done!
2008/08/18 Done!
2008/08/19 Done!
2008/08/20 Done!
2008/08/21 Done!
2008/08/22 Done!
2008/08/25 Done!
2008/08/26 Done!
2008/08/27 Done!
2008/08/28 Done!
2008/08/29 Done!
2008/09/02 Done!
2008/09/03 Done!
2008/09/04 Done!
2008/09/05 Done!
2008/09/08 Done!
2008/09/09 Done!
2008/09/10 Done!
2008/09/11 Done!
2008/09/12 Done!
2008/09/15 Done!
2008/09/16 Done!
2008/09/17 Done!
2008/09/18 Done!
2008/09/19 Done!
2008/09/22 Done!
2008/09/23 Done!
2008/09/24 Done!
2008/09/25 Done!
2008/09/26 Done!
2008/09/29 Done!
2008/09/30 Done!
2008/10/01 Done!
2008/10/02 Done!
2008/10/03 Done!
2008/10/06 Done!
2008/10/07 Done!
2008/10/08 Done!
2008/10/09 Done!
2008/10/10 Done!
2008/10/13 Done!
2008/10/14 Done!
2008/10/15 Done!
2008/10/16 Done!
2008/10/17 Done!
2008/10/20 Done!
2008/10/21 Done!
2008/10/22 Done!
2008/10/23 Done!
2008/10/24 Done!
2008/10/27 Done!
2008/10/28 Done!
2008/10/29 Done!
2008/10/30 Don

2011/04/05 Done!
2011/04/06 Done!
2011/04/07 Done!
2011/04/08 Done!
2011/04/11 Done!
2011/04/12 Done!
2011/04/13 Done!
2011/04/14 Done!
2011/04/15 Done!
2011/04/18 Done!
2011/04/19 Done!
2011/04/20 Done!
2011/04/21 Done!
2011/04/25 Done!
2011/04/26 Done!
2011/04/27 Done!
2011/04/28 Done!
2011/04/29 Done!
2011/05/02 Done!
2011/05/03 Done!
2011/05/04 Done!
2011/05/05 Done!
2011/05/06 Done!
2011/05/09 Done!
2011/05/10 Done!
2011/05/11 Done!
2011/05/12 Done!
2011/05/13 Done!
2011/05/16 Done!
2011/05/17 Done!
2011/05/18 Done!
2011/05/19 Done!
2011/05/20 Done!
2011/05/23 Done!
2011/05/24 Done!
2011/05/25 Done!
2011/05/26 Done!
2011/05/27 Done!
2011/05/31 Done!
2011/06/01 Done!
2011/06/02 Done!
2011/06/03 Done!
2011/06/06 Done!
2011/06/07 Done!
2011/06/08 Done!
2011/06/09 Done!
2011/06/10 Done!
2011/06/13 Done!
2011/06/14 Done!
2011/06/15 Done!
2011/06/16 Done!
2011/06/17 Done!
2011/06/20 Done!
2011/06/21 Done!
2011/06/22 Done!
2011/06/23 Done!
2011/06/24 Done!
2011/06/27 Done!
2011/06/28 Don

2013/09/20 Done!
2013/09/23 Done!
2013/09/24 Done!
2013/09/25 Done!
2013/09/26 Done!
2013/09/27 Done!
2013/09/30 Done!
2013/10/01 Done!
2013/10/02 Done!
2013/10/03 Done!
2013/10/04 Done!
2013/10/07 Done!
2013/10/08 Done!
2013/10/09 Done!
2013/10/10 Done!
2013/10/11 Done!
2013/10/14 Done!
2013/10/15 Done!
2013/10/16 Done!
2013/10/17 Done!
2013/10/18 Done!
2013/10/21 Done!
2013/10/22 Done!
2013/10/23 Done!
2013/10/24 Done!
2013/10/25 Done!
2013/10/28 Done!
2013/10/29 Done!
2013/10/30 Done!
2013/10/31 Done!
2013/11/01 Done!
2013/11/04 Done!
2013/11/05 Done!
2013/11/06 Done!
2013/11/07 Done!
2013/11/08 Done!
2013/11/11 Done!
2013/11/12 Done!
2013/11/13 Done!
2013/11/14 Done!
2013/11/15 Done!
2013/11/18 Done!
2013/11/19 Done!
2013/11/20 Done!
2013/11/21 Done!
2013/11/22 Done!
2013/11/25 Done!
2013/11/26 Done!
2013/11/27 Done!
2013/11/29 Done!
2013/12/02 Done!
2013/12/03 Done!
2013/12/04 Done!
2013/12/05 Done!
2013/12/06 Done!
2013/12/09 Done!
2013/12/10 Done!
2013/12/11 Done!
2013/12/12 Don

2015/10/28 Done!
2015/10/29 Done!
2015/10/30 Done!
2015/11/02 Done!
2015/11/03 Done!
2015/11/04 Done!
2015/11/05 Done!
2015/11/06 Done!
2015/11/09 Done!
2015/11/10 Done!
2015/11/11 Done!
2015/11/12 Done!
2015/11/13 Done!
2015/11/16 Done!
2015/11/17 Done!
2015/11/18 Done!
2015/11/19 Done!
2015/11/20 Done!
2015/11/23 Done!
2015/11/24 Done!
2015/11/25 Done!
2015/11/27 Done!
2015/11/30 Done!
2015/12/01 Done!
2015/12/02 Done!
2015/12/03 Done!
2015/12/04 Done!
2015/12/07 Done!
2015/12/08 Done!
2015/12/09 Done!
2015/12/10 Done!
2015/12/11 Done!
2015/12/14 Done!
2015/12/15 Done!
2015/12/16 Done!
2015/12/17 Done!
2015/12/18 Done!
2015/12/21 Done!
2015/12/22 Done!
2015/12/23 Done!
2015/12/24 Done!
2015/12/28 Done!
2015/12/29 Done!
2015/12/30 Done!
2015/12/31 Done!
2016/01/04 Done!
2016/01/05 Done!
2016/01/06 Done!
2016/01/07 Done!
2016/01/08 Done!
2016/01/11 Done!
2016/01/12 Done!
2016/01/13 Done!
2016/01/14 Done!
2016/01/15 Done!
2016/01/19 Done!
2016/01/20 Done!
2016/01/21 Done!
2016/01/22 Don

In [26]:
print(date_BOW.keys()); print()
# print(date_BOW['16/03/2016'].keys())
#If seeing nan, use the box above

dict_keys(['2008/08/08', '2008/08/11', '2008/08/12', '2008/08/13', '2008/08/14', '2008/08/15', '2008/08/18', '2008/08/19', '2008/08/20', '2008/08/21', '2008/08/22', '2008/08/25', '2008/08/26', '2008/08/27', '2008/08/28', '2008/08/29', '2008/09/02', '2008/09/03', '2008/09/04', '2008/09/05', '2008/09/08', '2008/09/09', '2008/09/10', '2008/09/11', '2008/09/12', '2008/09/15', '2008/09/16', '2008/09/17', '2008/09/18', '2008/09/19', '2008/09/22', '2008/09/23', '2008/09/24', '2008/09/25', '2008/09/26', '2008/09/29', '2008/09/30', '2008/10/01', '2008/10/02', '2008/10/03', '2008/10/06', '2008/10/07', '2008/10/08', '2008/10/09', '2008/10/10', '2008/10/13', '2008/10/14', '2008/10/15', '2008/10/16', '2008/10/17', '2008/10/20', '2008/10/21', '2008/10/22', '2008/10/23', '2008/10/24', '2008/10/27', '2008/10/28', '2008/10/29', '2008/10/30', '2008/10/31', '2008/11/03', '2008/11/04', '2008/11/05', '2008/11/06', '2008/11/07', '2008/11/10', '2008/11/11', '2008/11/12', '2008/11/13', '2008/11/14', '2008/11/

In [27]:
# TextBlob sentiment analysis

In [28]:
from textblob import TextBlob
#https://textblob.readthedocs.io/en/dev/quickstart.html 
# testimonial = TextBlob("Textblob is amazingly simple to use. What great fun!")
testimonial = TextBlob("great")
testimonial.sentiment

Sentiment(polarity=0.8, subjectivity=0.75)

In [29]:
Score_daily = {}
for date in date_BOW.keys():
    polarity_score_pos = 0; polarity_score_neg = 0; polarity_score_neu = 0; subjective_score = 0
    pos_word_num = 0; neg_word_num = 0; neu_word_num = 0;
    for word, count in date_BOW[date].items():
        testimonial = TextBlob(word)
        raw_sent_score, raw_subject_score = testimonial.sentiment
#         raw_subject_score = testimonial.sentiment[1]
        if raw_sent_score > 0:
            pos_word_num += count
            polarity_score_pos += raw_sent_score * count
        if raw_sent_score < 0:
            neg_word_num += count
            polarity_score_neg += raw_sent_score * count
        if raw_sent_score == 0:
            neu_word_num += count
            polarity_score_neu += raw_sent_score * count
        
        subjective_score += raw_subject_score * count
        Score_daily[date] = (polarity_score_pos, polarity_score_neg, polarity_score_neu,subjective_score, pos_word_num, neg_word_num, neu_word_num)
    print(date, 'Done!')

2008/08/08 Done!
2008/08/11 Done!
2008/08/12 Done!
2008/08/13 Done!
2008/08/14 Done!
2008/08/15 Done!
2008/08/18 Done!
2008/08/19 Done!
2008/08/20 Done!
2008/08/21 Done!
2008/08/22 Done!
2008/08/25 Done!
2008/08/26 Done!
2008/08/27 Done!
2008/08/28 Done!
2008/08/29 Done!
2008/09/02 Done!
2008/09/03 Done!
2008/09/04 Done!
2008/09/05 Done!
2008/09/08 Done!
2008/09/09 Done!
2008/09/10 Done!
2008/09/11 Done!
2008/09/12 Done!
2008/09/15 Done!
2008/09/16 Done!
2008/09/17 Done!
2008/09/18 Done!
2008/09/19 Done!
2008/09/22 Done!
2008/09/23 Done!
2008/09/24 Done!
2008/09/25 Done!
2008/09/26 Done!
2008/09/29 Done!
2008/09/30 Done!
2008/10/01 Done!
2008/10/02 Done!
2008/10/03 Done!
2008/10/06 Done!
2008/10/07 Done!
2008/10/08 Done!
2008/10/09 Done!
2008/10/10 Done!
2008/10/13 Done!
2008/10/14 Done!
2008/10/15 Done!
2008/10/16 Done!
2008/10/17 Done!
2008/10/20 Done!
2008/10/21 Done!
2008/10/22 Done!
2008/10/23 Done!
2008/10/24 Done!
2008/10/27 Done!
2008/10/28 Done!
2008/10/29 Done!
2008/10/30 Don

2010/07/09 Done!
2010/07/12 Done!
2010/07/13 Done!
2010/07/14 Done!
2010/07/15 Done!
2010/07/16 Done!
2010/07/19 Done!
2010/07/20 Done!
2010/07/21 Done!
2010/07/22 Done!
2010/07/23 Done!
2010/07/26 Done!
2010/07/27 Done!
2010/07/28 Done!
2010/07/29 Done!
2010/07/30 Done!
2010/08/02 Done!
2010/08/03 Done!
2010/08/04 Done!
2010/08/05 Done!
2010/08/06 Done!
2010/08/09 Done!
2010/08/10 Done!
2010/08/11 Done!
2010/08/12 Done!
2010/08/13 Done!
2010/08/16 Done!
2010/08/17 Done!
2010/08/18 Done!
2010/08/19 Done!
2010/08/20 Done!
2010/08/23 Done!
2010/08/24 Done!
2010/08/25 Done!
2010/08/26 Done!
2010/08/27 Done!
2010/08/30 Done!
2010/08/31 Done!
2010/09/01 Done!
2010/09/02 Done!
2010/09/03 Done!
2010/09/07 Done!
2010/09/08 Done!
2010/09/09 Done!
2010/09/10 Done!
2010/09/13 Done!
2010/09/14 Done!
2010/09/15 Done!
2010/09/16 Done!
2010/09/17 Done!
2010/09/20 Done!
2010/09/21 Done!
2010/09/22 Done!
2010/09/23 Done!
2010/09/24 Done!
2010/09/27 Done!
2010/09/28 Done!
2010/09/29 Done!
2010/09/30 Don

2012/06/07 Done!
2012/06/08 Done!
2012/06/11 Done!
2012/06/12 Done!
2012/06/13 Done!
2012/06/14 Done!
2012/06/15 Done!
2012/06/18 Done!
2012/06/19 Done!
2012/06/20 Done!
2012/06/21 Done!
2012/06/22 Done!
2012/06/25 Done!
2012/06/26 Done!
2012/06/27 Done!
2012/06/28 Done!
2012/06/29 Done!
2012/07/02 Done!
2012/07/03 Done!
2012/07/05 Done!
2012/07/06 Done!
2012/07/09 Done!
2012/07/10 Done!
2012/07/11 Done!
2012/07/12 Done!
2012/07/13 Done!
2012/07/16 Done!
2012/07/17 Done!
2012/07/18 Done!
2012/07/19 Done!
2012/07/20 Done!
2012/07/23 Done!
2012/07/24 Done!
2012/07/25 Done!
2012/07/26 Done!
2012/07/27 Done!
2012/07/30 Done!
2012/07/31 Done!
2012/08/01 Done!
2012/08/02 Done!
2012/08/03 Done!
2012/08/06 Done!
2012/08/07 Done!
2012/08/08 Done!
2012/08/09 Done!
2012/08/10 Done!
2012/08/13 Done!
2012/08/14 Done!
2012/08/15 Done!
2012/08/16 Done!
2012/08/17 Done!
2012/08/20 Done!
2012/08/21 Done!
2012/08/22 Done!
2012/08/23 Done!
2012/08/24 Done!
2012/08/27 Done!
2012/08/28 Done!
2012/08/29 Don

2014/05/09 Done!
2014/05/12 Done!
2014/05/13 Done!
2014/05/14 Done!
2014/05/15 Done!
2014/05/16 Done!
2014/05/19 Done!
2014/05/20 Done!
2014/05/21 Done!
2014/05/22 Done!
2014/05/23 Done!
2014/05/27 Done!
2014/05/28 Done!
2014/05/29 Done!
2014/05/30 Done!
2014/06/02 Done!
2014/06/03 Done!
2014/06/04 Done!
2014/06/05 Done!
2014/06/06 Done!
2014/06/09 Done!
2014/06/10 Done!
2014/06/11 Done!
2014/06/12 Done!
2014/06/13 Done!
2014/06/16 Done!
2014/06/17 Done!
2014/06/18 Done!
2014/06/19 Done!
2014/06/20 Done!
2014/06/23 Done!
2014/06/24 Done!
2014/06/25 Done!
2014/06/26 Done!
2014/06/27 Done!
2014/06/30 Done!
2014/07/01 Done!
2014/07/02 Done!
2014/07/03 Done!
2014/07/07 Done!
2014/07/08 Done!
2014/07/09 Done!
2014/07/10 Done!
2014/07/11 Done!
2014/07/14 Done!
2014/07/15 Done!
2014/07/16 Done!
2014/07/17 Done!
2014/07/18 Done!
2014/07/21 Done!
2014/07/22 Done!
2014/07/23 Done!
2014/07/24 Done!
2014/07/25 Done!
2014/07/28 Done!
2014/07/29 Done!
2014/07/30 Done!
2014/07/31 Done!
2014/08/01 Don

2016/04/08 Done!
2016/04/11 Done!
2016/04/12 Done!
2016/04/13 Done!
2016/04/14 Done!
2016/04/15 Done!
2016/04/18 Done!
2016/04/19 Done!
2016/04/20 Done!
2016/04/21 Done!
2016/04/22 Done!
2016/04/25 Done!
2016/04/26 Done!
2016/04/27 Done!
2016/04/28 Done!
2016/04/29 Done!
2016/05/02 Done!
2016/05/03 Done!
2016/05/04 Done!
2016/05/05 Done!
2016/05/06 Done!
2016/05/09 Done!
2016/05/10 Done!
2016/05/11 Done!
2016/05/12 Done!
2016/05/13 Done!
2016/05/16 Done!
2016/05/17 Done!
2016/05/18 Done!
2016/05/19 Done!
2016/05/20 Done!
2016/05/23 Done!
2016/05/24 Done!
2016/05/25 Done!
2016/05/26 Done!
2016/05/27 Done!
2016/05/31 Done!
2016/06/01 Done!
2016/06/02 Done!
2016/06/03 Done!
2016/06/06 Done!
2016/06/07 Done!
2016/06/08 Done!
2016/06/09 Done!
2016/06/10 Done!
2016/06/13 Done!
2016/06/14 Done!
2016/06/15 Done!
2016/06/16 Done!
2016/06/17 Done!
2016/06/20 Done!
2016/06/21 Done!
2016/06/22 Done!
2016/06/23 Done!
2016/06/24 Done!
2016/06/27 Done!
2016/06/28 Done!
2016/06/29 Done!
2016/06/30 Don

In [30]:
len(Score_daily)

1989

In [31]:
# Sentiment_dict = {}
# for date in date_BOW.keys():
#     testimonial = TextBlob[date_BOWp[date]]

In [32]:
Score_daily #Date-Polarity-Subjectivity

{'2008/08/08': (1.6863636363636365,
  -2.914285714285714,
  0.0,
  6.228354978354979,
  7,
  9,
  246),
 '2008/08/11': (2.8827272727272724, -0.85, 0.0, 5.532424242424243, 10, 2, 174),
 '2008/08/12': (2.4006493506493505,
  -2.4388888888888887,
  0.0,
  9.16962481962482,
  9,
  8,
  215),
 '2008/08/13': (1.7857142857142856,
  -2.180555555555556,
  0.0,
  7.302380952380952,
  6,
  9,
  214),
 '2008/08/14': (2.659577922077922,
  -1.9333333333333333,
  0.0,
  8.752380952380951,
  10,
  7,
  183),
 '2008/08/15': (1.6166666666666665, -1.7, 0.0, 8.233333333333334, 7, 7, 217),
 '2008/08/18': (2.9287878787878787,
  -4.2875,
  0.0,
  12.346969696969696,
  9,
  8,
  167),
 '2008/08/19': (1.253030303030303,
  -2.9499999999999997,
  0.0,
  7.321212121212121,
  5,
  7,
  186),
 '2008/08/20': (0.8727272727272727,
  -0.5666666666666667,
  0.0,
  4.087878787878788,
  5,
  4,
  220),
 '2008/08/21': (2.7, -1.75, 0.0, 6.1000000000000005, 9, 8, 225),
 '2008/08/22': (1.897727272727273, -0.75, 0.0, 5.73409090

In [33]:
pd_Score_daily = pd.DataFrame.from_dict(Score_daily) #Date-Polarity-Subjectivity->DataFrame

In [38]:
pd_Score_daily_T = pd_Score_daily.T
(pd_Score_daily.head())


,2008/08/08,2008/08/11,2008/08/12,2008/08/13,2008/08/14,2008/08/15,2008/08/18,2008/08/19,2008/08/20,2008/08/21,...,2016/06/20,2016/06/21,2016/06/22,2016/06/23,2016/06/24,2016/06/27,2016/06/28,2016/06/29,2016/06/30,2016/07/01
0,1.686364,2.882727,2.400649,1.785714,2.659578,1.616667,2.928788,1.253030,0.872727,2.70,...,4.706061,4.635227,2.909091,2.789394,2.882619,3.622727,4.650758,5.179870,3.397619,3.016667
1,-2.914286,-0.850000,-2.438889,-2.180556,-1.933333,-1.700000,-4.287500,-2.950000,-0.566667,-1.75,...,-3.841667,-2.604762,-2.426984,-1.196429,-2.571429,-4.216667,-3.100000,-3.725000,-3.738889,-3.855556
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,6.228355,5.532424,9.169625,7.302381,8.752381,8.233333,12.346970,7.321212,4.087879,6.10,...,13.267424,13.606710,9.569589,9.198377,10.832857,13.284091,13.180303,14.772186,10.476984,10.455556
4,7.000000,10.000000,9.000000,6.000000,10.000000,7.000000,9.000000,5.000000,5.000000,9.00,...,11.000000,12.000000,10.000000,9.000000,9.000000,8.000000,13.000000,14.000000,9.000000,8.000000


In [45]:
#(polarity_score_pos, polarity_score_neg, polarity_score_neu,subjective_score)
# (pos_word_num, neg_word_num, neu_word_num)
label_add = ['pos', 'neg', 'neu','subjectivity','pos_count', 'neg_count', 'neu_count']
pd_Score_daily_T.head()

,polarity_score_pos,polarity_score_neg,polarity_score_neu,subjective_score,pos_word_num,neg_word_num,neu_word_num
2008/08/08,1.686364,-2.914286,0.0,6.228355,7.0,9.0,246.0
2008/08/11,2.882727,-0.850000,0.0,5.532424,10.0,2.0,174.0
2008/08/12,2.400649,-2.438889,0.0,9.169625,9.0,8.0,215.0
2008/08/13,1.785714,-2.180556,0.0,7.302381,6.0,9.0,214.0
2008/08/14,2.659578,-1.933333,0.0,8.752381,10.0,7.0,183.0


In [46]:
pd_Score_daily_T.columns = label_add
pd_Score_daily_T.head()

,pos,neg,neu,subjectivity,pos_count,neg_count,neu_count
2008/08/08,1.686364,-2.914286,0.0,6.228355,7.0,9.0,246.0
2008/08/11,2.882727,-0.850000,0.0,5.532424,10.0,2.0,174.0
2008/08/12,2.400649,-2.438889,0.0,9.169625,9.0,8.0,215.0
2008/08/13,1.785714,-2.180556,0.0,7.302381,6.0,9.0,214.0
2008/08/14,2.659578,-1.933333,0.0,8.752381,10.0,7.0,183.0


In [47]:
#By TextBlob sentiment
pd_Score_daily_T.to_csv('8_yr_news_sentiment_score.csv')

In [24]:
# xs = range(0,len(date_BOW['17/03/2016']))
# figure(figsize=(16, 7), dpi=80, facecolor='w', edgecolor='k')
# barplt_col_sum = plt.bar(xs, date_BOW['17/03/2016'].values(), color='g', width = 0.8, align = 'center')
# plt.xticks(rotation=-90)
# # plt.yticks(np.arange(0 , 15000, step = 1000))
# plt.title('Unique word count on the 17/03/2016')
# plt.xlabel('Date')
# plt.ylabel('Word count')
# plt.savefig('date_volumne4_2.pdf')

In [25]:
syns = wordnet.synsets('program')
#Synset
print(syns[0].name())
#Definition
print(syns[0].definition())
#Example
print(syns[0].examples())


plan.n.01
a series of steps to be carried out or goals to be accomplished
['they drew up a six-step plan', 'they discussed plans for a new bond issue']


In [26]:
#Synonymns
word = 'good'

def Synonymns_gen(word):
    Synonymns=[]
    Antonyms=[]
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
#             print('l',l) #Show all lemmas, try 'good'
            Synonymns.append(l.name())
            if l.antonyms():
                Antonyms.append(l.antonyms()[0].name())
    return (Synonymns, Antonyms)
                
(Synonymns, Antonyms) = Synonymns_gen(word)           
print(set(Synonymns))
print(set(Antonyms))

{'adept', 'full', 'safe', 'in_effect', 'respectable', 'just', 'soundly', 'ripe', 'proficient', 'near', 'secure', 'undecomposed', 'upright', 'good', 'well', 'in_force', 'serious', 'dear', 'practiced', 'goodness', 'effective', 'trade_good', 'beneficial', 'sound', 'expert', 'estimable', 'thoroughly', 'honest', 'skillful', 'salutary', 'skilful', 'commodity', 'unspoiled', 'honorable', 'dependable', 'right', 'unspoilt'}
{'evilness', 'bad', 'ill', 'evil', 'badness'}


In [27]:
#Word similarity

def compute_word_sim(W1, W2):
    sim = (W1.wup_similarity(W2))
    return sim

W1 = wordnet.synsets('ship')[0]
W2 = wordnet.synsets('boat')[0]
sim1 = compute_word_sim(W1, W2)
print(sim1)
W3 = wordnet.synsets('cat')[0]
W4 = wordnet.synsets('dog')[0]
sim2 = compute_word_sim(W3, W4)
print(sim2)
W5 = wordnet.synsets('Black')[0]
W6 = wordnet.synsets('back')[0]
sim3 = compute_word_sim(W5, W6)
print(sim3)
W7 = wordnet.synsets('radiation')[0]
W8 = wordnet.synsets('light')[0]
sim4 = compute_word_sim(W7, W8)
print(sim4)
W9 = wordnet.synsets('lesbian')[0]
W10 = wordnet.synsets('gay')[0]
sim3 = compute_word_sim(W9, W10)
print(sim3)
W11 = wordnet.synsets('finger')[0]
W12 = wordnet.synsets('hand')[0]
sim4 = compute_word_sim(W11, W12)
print(sim4)

0.9090909090909091
0.8571428571428571
0.14285714285714285
0.8421052631578947
0.7058823529411765
0.875


In [28]:
print(syns[0].lemmas())
#Just name
print(syns[0].lemmas()[0].name())

[Lemma('plan.n.01.plan'), Lemma('plan.n.01.program'), Lemma('plan.n.01.programme')]
plan


In [29]:
from nltk.stem import *
from nltk.corpus import sentiwordnet as swn
# SentiNet 
#https://stackoverflow.com/questions/33959459/usage-of-nltk-sentiwordnet-with-python
# Sentiment Symposium Tutorial: Lexicons (Useful guide)
# http://sentiment.christopherpotts.net/lexicons.html#resources
# http://ontotext.fbk.eu/sentiwn.html

In [77]:
W2 = list(swn.senti_synsets('sad'))
print(W2)
w2_0 = W2[0]
w2_1 = W2[1]
w2_2 = W2[2]

[SentiSynset('sad.a.01'), SentiSynset('sad.s.02'), SentiSynset('deplorable.s.01')]


In [78]:
W1 = list(swn.senti_synsets('good'))
print(W1[:5])
w1_0 = W1[0]
w1_1 = W1[1]
w1_2 = W1[2]

[SentiSynset('good.n.01'), SentiSynset('good.n.02'), SentiSynset('good.n.03'), SentiSynset('commodity.n.01'), SentiSynset('good.a.01')]


In [81]:
print(); print(w1_0); print(w1_0)
print('pos: ', w1_0.pos_score())
print('neg: ', w1_0.neg_score())

print(); print(w1_1); print(W1_1)
print('pos: ', w1_1.pos_score())
print('neg: ', w1_1.neg_score())

print(); print(w1_2); print(w1_2)
print('pos: ', w1_2.pos_score())
print('neg: ', w1_2.neg_score())


<good.n.01: PosScore=0.5 NegScore=0.0>
<good.n.01: PosScore=0.5 NegScore=0.0>
pos:  0.5
neg:  0.0

<good.n.02: PosScore=0.875 NegScore=0.0>
<good.n.02: PosScore=0.875 NegScore=0.0>
pos:  0.875
neg:  0.0

<good.n.03: PosScore=0.625 NegScore=0.0>
<good.n.03: PosScore=0.625 NegScore=0.0>
pos:  0.625
neg:  0.0


In [82]:
print(); print(w2_0); print(w2_0)
print('pos: ', w2_0.pos_score())
print('neg: ', w2_0.neg_score())

print(); print(w2_1); print(w2_1)
print('pos: ', w2_1.pos_score())
print('neg: ', w2_1.neg_score())

print(); print(w2_2); print(w2_2)
print('pos: ', w2_2.pos_score())
print('neg: ', w2_2.neg_score())


<sad.a.01: PosScore=0.125 NegScore=0.75>
<sad.a.01: PosScore=0.125 NegScore=0.75>
pos:  0.125
neg:  0.75

<sad.s.02: PosScore=0.0 NegScore=0.25>
<sad.s.02: PosScore=0.0 NegScore=0.25>
pos:  0.0
neg:  0.25

<deplorable.s.01: PosScore=0.0 NegScore=1.0>
<deplorable.s.01: PosScore=0.0 NegScore=1.0>
pos:  0.0
neg:  1.0


In [ ]:
#OpinionFinder-

